## Install the Required Packages

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install diffusers==0.4.1
!pip install accelerate
!pip install torchvision
!pip install transformers>=4.21.0
!pip install ftfy
!pip install tensorboard
!pip install modelcards
!pip install datasets
!pip install -U diffusers

## Import the Required Packages

In [ ]:
import datasets
import shutil
import h5py
import numpy as np
import io
import PIL.Image as Image
from torchvision.utils import save_image
import torch
import json
from datasets import load_dataset
import pickle
import torch
from diffusers import StableDiffusionPipeline

/ext3/miniconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Check GPU Information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

## Initialize the accelerate environment

In [ ]:
!accelerate config default

Configuration already exists at /home/sv2128/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


##Hugging Face Hub Login

You need to accept the model license before downloading or using the weights. In this example we’ll use model version v1-4, so you’ll need to visit its card (https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree.

You have to be a registered user in 🤗 Hugging Face Hub, and you’ll also need to use an access token for the code to work. For more information on access tokens, please refer to this section of the documentation (https://huggingface.co/docs/hub/security-tokens).

In [ ]:
from huggingface_hub import login
login(token='your_access_token')

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /home/sv2128/.huggingface/token
Login successful


##Function to Construct ROCO Dataset as Required by Stable Diffusion
To run on your own training files you need to prepare the dataset according to the format required by datasets. You can upload your dataset to the Hugging Face Hub, or you can prepare a local folder with your files. This documentation (https://huggingface.co/docs/hub/security-tokens) explains how to do it.

Make sure there is an empty train folder in ROCO_DIR. Download the ROCO dataset from this link (https://github.com/razorx89/roco-dataset) and keep it inside ROCO_DIR.


In [ ]:
"""

This funtion uses only the training and validation splits of ROCO dataset 
and converts them into the required format for fine-tuning of the Stable 
Diffusion model.

"""

def construct_roco(ROCO_DIR):
  metajson_list=[]

  file1= open(ROCO_DIR+'/roco-dataset-master/data/train/radiology/captions.txt', 'r')
  captions_train_rad = file1.readlines()
  for i in range(len(captions_train_rad)):
    temp_meta={}
    temp_meta['file_name']=captions_train_rad[i].split('\t')[0]+'.jpg'
    temp_meta['text']=captions_train_rad[i].split('\t')[1]
    metajson_list.append(temp_meta)

  file2= open(ROCO_DIR+'/roco-dataset-master/data/train/non-radiology/captions.txt', 'r')
  captions_train_nonrad = file2.readlines()
  for i in range(len(captions_train_nonrad)):
    temp_meta={}
    temp_meta['file_name']=captions_train_nonrad[i].split('\t')[0]+'.jpg'
    temp_meta['text']=captions_train_nonrad[i].split('\t')[1]
    metajson_list.append(temp_meta)

  file3= open(ROCO_DIR+'/roco-dataset-master/data/validation/radiology/captions.txt', 'r')
  captions_val_rad = file3.readlines()
  for i in range(len(captions_val_rad)):
    temp_meta={}
    temp_meta['file_name']=captions_val_rad[i].split('\t')[0]+'.jpg'
    temp_meta['text']=captions_val_rad[i].split('\t')[1]
    metajson_list.append(temp_meta)

  file4= open(ROCO_DIR+'/roco-dataset-master/data/validation/non-radiology/captions.txt', 'r')
  captions_val_nonrad = file4.readlines()
  for i in range(len(captions_val_nonrad)):
    temp_meta={}
    temp_meta['file_name']=captions_val_nonrad[i].split('\t')[0]+'.jpg'
    temp_meta['text']=captions_val_nonrad[i].split('\t')[1]
    metajson_list.append(temp_meta)

  print(metajson_list)
  with open(ROCO_DIR+"/train/metadata.jsonl", 'w') as f:
    for item in metajson_list:
        f.write(json.dumps(item) + "\n")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
ROCO_DIR = "path/to/ROCO_DIR"

In [ ]:
construct_roco(ROCO_DIR)

In [ ]:
# roco_dataset = datasets.load_dataset("imagefolder", data_dir=ROCO_DIR, split="train")
# print(roco_dataset[0])

## Fine-tune Stable Diffusion

The fine-tuning of Stable Diffusion takes around 33 hours on V100 GPU for 10 epochs. Please tune the hyper-parameters based on your requirements.

In [ ]:
!(accelerate launch HF_StableD_ML/train_text_to_image.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --train_data_dir="path/to/ROCO_DIR/train" \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=16 \
  --gradient_accumulation_steps=8 \
  --gradient_checkpointing \
  --mixed_precision="bf16" \
  --num_train_epochs=5 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="path/to/ROCO_DIR/output/directory" \
)

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/diffusers/utils/deprecation_utils.py:35: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  warnings.warn(warning + message, FutureWarning)
Resolving data files: 100%|████████████| 79094/79094 [00:07<00:00, 10481.49it/s]
12/09/2022 14:15:31 - WARNING - datasets.builder - Using custom data configuration default-5eca8a2d2a0bd9a9
12/09/2022 14:15:31 - WARNING - datasets.builder - Reusing dataset imagefolder (/home/sv2128/.cache/huggingface/datasets/imagefolder/default-5eca8a2d2a0bd9a9/0.0.0/0fc50c79b681877cc46b23245a6ef5333d036f48db40d53765a68034bc48faff)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]
12

## Visualize Epoch Vs Loss Graph 

In [ ]:
!tensorboard --logdir path/to/ROCO_DIR/output/directory

## Function to Generate ROCO Dataset

This function generates images based on the captions as text prompts present in test set of ROCO

Make sure there is an empty test folder in ROCO_DIR location

In [ ]:
def generate_roco(ROCO_DIR):
    
  model_path = ROCO_DIR+"path/to/ROCO_DIR/output/directory"
  pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
  pipe.safety_checker = lambda images, clip_input: (images, False)
  pipe.to("cuda")


  file1= open(ROCO_DIR+'/roco-dataset-master/data/test/radiology/captions.txt', 'r')
  captions_test_rad = file1.readlines()
  for i in range(len(captions_test_rad)):
    
    print(i)
    
    image = pipe(prompt=captions_test_rad[i].split('\t')[1]).images[0]
    image.save(ROCO_DIR+"/test/"+captions_test_rad[i].split('\t')[0]+'.jpg')

In [ ]:
generate_roco(ROCO_DIR)

0


100%|██████████| 50/50 [00:05<00:00,  8.57it/s]


1


100%|██████████| 50/50 [00:01<00:00, 27.12it/s]


2


...

100%|██████████| 50/50 [00:01<00:00, 26.67it/s]


8177


100%|██████████| 50/50 [00:01<00:00, 26.65it/s]


8178


100%|██████████| 50/50 [00:01<00:00, 26.64it/s]
